20L-1149     Yousaf Khalid    BCS 7B
20L-0942     Fahad Azhar      BCS 7A
20L-1079     Umer Waheed      BCS 7A

In [12]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np

def upload_image():
    global file_path, img_label, process_button
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((300, 300))
        img = ImageTk.PhotoImage(img)

        if img_label is None:
            img_label = tk.Label(root, image=img)
            img_label.image = img
            img_label.pack(side='top', padx=10, pady=10)
        else:
            img_label.configure(image=img)
            img_label.image = img

        if process_button is None:
            process_button = tk.Button(root, text="Process for Edge Detection and Hough Transformation", command=process_image)
            process_button.pack(side='bottom', padx=10, pady=10)

def edge_detection(image_path):
    img = cv2.imread(image_path, 0)
    edges = cv2.Canny(img, 50, 150, apertureSize=3)
    return edges

def hough_transform(edges):
    height, width = edges.shape
    diag_len = int(np.sqrt(width ** 2 + height ** 2))
    thetas = np.deg2rad(np.arange(0, 180))
    rhos = np.linspace(-diag_len, diag_len, diag_len * 2)
    accumulator = np.zeros((2 * diag_len, len(thetas)), dtype=np.uint64)
    edge_points = np.argwhere(edges != 0)
    for y, x in edge_points:
        for theta_idx, theta in enumerate(thetas):
            rho = int((x * np.cos(theta)) + (y * np.sin(theta))) + diag_len
            accumulator[rho, theta_idx] += 1
    return accumulator, thetas, rhos

def draw_detected_lines(image, accumulator, thetas, rhos, threshold=1000):
    for y in range(accumulator.shape[0]):
        for x in range(accumulator.shape[1]):
            if accumulator[y, x] > threshold:
                rho = rhos[y]
                theta = thetas[x]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = (a * rho)
                y0 = (b * rho)
                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return image

def process_image():
    try:
        edges = edge_detection(file_path)
        accumulator, thetas, rhos = hough_transform(edges)
        original_image = cv2.imread(file_path)
        lines_image = draw_detected_lines(original_image, accumulator, thetas, rhos, threshold=100)
        edge_image_pil = Image.fromarray(edges)
        lines_image_pil = Image.fromarray(cv2.cvtColor(lines_image, cv2.COLOR_BGR2RGB))
        edge_image_pil = ImageTk.PhotoImage(edge_image_pil)
        edge_window = tk.Toplevel(root)
        edge_window.title("Edge Detected Image")
        edge_label = tk.Label(edge_window, image=edge_image_pil)
        edge_label.image = edge_image_pil
        edge_label.pack()
        lines_image_pil = ImageTk.PhotoImage(lines_image_pil)
        lines_window = tk.Toplevel(root)
        lines_window.title("Detected Lines")
        lines_label = tk.Label(lines_window, image=lines_image_pil)
        lines_label.image = lines_image_pil
        lines_label.pack()
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

root = tk.Tk()
root.title("Image Processing with Hough Transform")
file_path = None
img_label = None
process_button = None
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(side='top', padx=10, pady=10)
root.mainloop()
